**PydanticOuputParser를 사용하면 상세한 출력이 가능하다.**

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

**PydanticOuputParser를 사용하지 않았을 때 (기본 버전)**

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

chain = prompt | llm

output = chain.invoke({"email_conversation": email_conversation})
output

AIMessage(content='- ZENESIS 모델에 대한 상세한 브로슈어 요청\n- 유통 전략과 마케팅 계획을 구체화하기 위한 협력 논의\n- 다음 주 화요일(1월 15일) 오전 10시에 미팅 제안', response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 479, 'total_tokens': 566}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6a0828ab-c221-489f-a531-e8cc10a89c6c-0', usage_metadata={'input_tokens': 479, 'output_tokens': 87, 'total_tokens': 566})

**PydanticOuputParser를 사용할 경우**

In [7]:


from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
chain = prompt | llm

output = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)



In [8]:
output

AIMessage(content='```json\n{\n  "person": "김철수",\n  "email": "chulsoo.kim@bikecorporation.me",\n  "subject": "\\"ZENESIS\\" 자전거 유통 협력 및 미팅 일정 제안",\n  "summary": "바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 \'ZENESIS\' 모델에 대한 상세한 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 오전 10시에 미팅을 제안함.",\n  "date": "1월 15일 오전 10시"\n}\n```', response_metadata={'token_usage': {'completion_tokens': 145, 'prompt_tokens': 733, 'total_tokens': 878}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None}, id='run-cd5fb584-74c3-4b60-b662-484fb0a0a6c4-0', usage_metadata={'input_tokens': 733, 'output_tokens': 145, 'total_tokens': 878})

In [ ]:
### 만약 출력 형태를 json이 아니라 Pydantic객체 형태로 얻고 싶다면 .with_structured_output(Pydantic) 를 사용하면 된다.
# llm_with_structered = ChatOpenAI(
#     temperature=0, model_name="gpt-4o"
# ).with_structured_output(EmailSummary)

#또는
#chain = prompt | llm | parser 마지막에 parser 체인을 걸어주면 된다.